In [1]:
import fitz  # PyMuPDF
import re
import zipfile
import os
import shutil
import pandas as pd
from pathlib import Path

In [28]:
import os
import glob

# Specify the folder containing the PDFs
pdf_folder = "POs"
pdf_folder_test ='PO_test'

# Get the absolute path of the folder
pdf_folder_path = os.path.join("/Users/luzhaolong/Desktop/po_parser", pdf_folder_test)

# List all PDF files in the folder
pdf_files = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

# Return the list of PDF file paths
pdf_files  # This will contain all the paths as a list

['/Users/luzhaolong/Desktop/po_parser/PO_test/PO106900.pdf',
 '/Users/luzhaolong/Desktop/po_parser/PO_test/PO106822.pdf',
 '/Users/luzhaolong/Desktop/po_parser/PO_test/PO106821.pdf',
 '/Users/luzhaolong/Desktop/po_parser/PO_test/PO106666.pdf']

In [29]:
pdf_path = pdf_files[2]

In [27]:
pdf_path

'/Users/luzhaolong/Desktop/po_parser/POs/PO 100350.pdf'

In [4]:
doc = fitz.open(pdf_path)
full_text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text_blocks = page.get_text("blocks")
    text_blocks.sort(key=lambda block: (block[1], block[0]))
    for block in text_blocks:
        full_text += block[4] + "\n"
doc.close()

In [5]:
full_text

'▌PURCHASE ORDER\nOriginal\n\nIndustrial Laser Machines, LLC\n\nDocument Number\nDocument Date\nPage\n\n4122 Metric Drive\nSuite 100\nWinter Park\n32792\nFL\n\n1/2\n06/19/2025\nPO100350\n\nCustomer No.\nFederal Tax ID - Business Partner\n\nV0281\n\nYour Reference\n\n250611-01\n\nYour Contact\n\nLINGBO GROUP INC\n\n-No Sales Employee-\n\n3333\nWARRENVILLE RD\nLISLE\n60532\nIL\nUS\nSTE 200\n\nDelivery Address\n\nIndustrial Laser Machines, LLC\n\n4122 Metric Drive\nSuite 100\nWinter Park\n32792\nFL\nUS\n\nCurrency: $\nDelivery Date:\n07/17/2025\n\nDescription\nQuantity\nUoM\nPrice\nTax %\n Total\nItem Details\nPrice\nTax %Tax %\n\nEach\n15.5000 \n0.0000\n155.00\nPLATE, SOLENOID & PCB MOUNT, \nSHUTTER\n001\n 10\nRev A\nPLATE, SOLENOID & PCB MOUNT, \nSHUTTER\n\nItem Code:\n102219\nItem Code:\nItem Code:\n\nDelivery Date:\n06/19/2025\n\nEach\n18.2500 \n0.0000\n182.50\nBRACKET, SENSOR PCB MOUNT, TOP \nADJUST, SHUTTER\n002\n 10\nRev A\nBRACKET, SENSOR PCB MOUNT, TOP \nADJUST, SHUTTER\n\nItem C

In [42]:
def extract_po_info(pdf_path: str) -> dict:
    """Extract purchase order header info from a single PDF."""
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text_blocks = page.get_text("blocks")
            text_blocks.sort(key=lambda block: (block[1], block[0]))
            for block in text_blocks:
                full_text += block[4] + "\n"
        doc.close()

        doc_number_match =  re.search(r"Purchase Order No\.\s*(\d+)", full_text)
        document_number = doc_number_match.group(1) if doc_number_match else ""

        reference_match = re.search(r"Your Reference\s+([\w\-\/]*)", full_text)
        reference = reference_match.group(1).strip() if reference_match else ""
        if reference.lower() == "your":
            reference = ""

        payment_match =  re.search(r"Payment Terms([\s\S]+?)(?:\n\n|\Z)", full_text)
        lines = [line.strip() for line in payment_match.group(1).splitlines() if line.strip()]
                # Look for the first line that is not "Buyer"
        payment_term = None
        for line in lines:
            if line.lower() != "buyer" and line.startswith("Net"):
                payment_term = line
                break

        doc_date_match = re.search(r"Date\s+([A-Za-z]+\s+\d{1,2},\s+\d{4})", full_text)
        document_date = doc_date_match.group(1) if doc_date_match else ""

        return {
            "Document Number": document_number,
            "Reference": reference,
            "Payment Term": payment_term,
            "Document Date": document_date,
        }

    except Exception as e:
        return {"error": str(e)} 

In [43]:
extract_po_info(pdf_path)

{'Document Number': '106821',
 'Reference': '',
 'Payment Term': 'Net 45 days',
 'Document Date': 'October 13, 2022'}

In [41]:
doc = fitz.open(pdf_path)
full_text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text_blocks = page.get_text("blocks")
    text_blocks.sort(key=lambda block: (block[1], block[0]))
    for block in text_blocks:
        full_text += block[4] + "\n"
doc.close()
# Remove commas in numbers (e.g., 1,000 → 1000)
full_text = re.sub(r'(?<=\d),(?=\d)', '', full_text)
lines = full_text.splitlines()
stop_marker = "Subtotal"
inside_table = False
blocks = []
current_block = []
#waiting_for_date = False
prev_line = ""
start_marker_found = False

for line in lines:
    line_strip = line.strip()
    if not inside_table:
        combined = (prev_line + line_strip).replace(" ", "")
        if combined == "Line Amount":
            inside_table = True
            prev_line = ""
            start_marker_found = True
            continue
        #if line_strip == "Tax %":
        #    prev_line = line_strip
        #    continue
        #prev_line = line_strip
        #continue
    if line_strip.startswith(stop_marker):
        break
    current_block.append(line_strip)
    blocks.append(parse_block_protech(current_block))
    current_block = []

In [43]:
full_text   

'Purchase Order\n\nPurchase Order No. 106821\n\nDate October 13, 2022\nPage 1\nPro-Tech Sales Company \nInc\n2802 Hitchcock ave Downers Grove, IL 60515\n\n800-766-TECH (8324)   (630) 271-0551    (630) 271-0661 Fax\n\nVendor\nShip-to Address\n\nLingbo Group Inc.\n2802 Hitchcock Ave.\n\nCarter Ding\n\n3333 Warrenville Rd. Suite 200\nDowners Grove, IL 60515\n\nLislie, IL 60532\nUSA\n\nShipment Method\nPayment Terms\nBuyer\nNet 45 days\n\nItem Number\nDescription\nReq. Date\nUofM\nOrdered\nUnit Cost\nLine Amount\nB9403-A\nFabricated Saddle (Tenova Core \n\nEach\n4\n28800.0\n\n115200.00\n\nILO1B9344-TS2)\n\n0\n\nMaterial: 310ss\n\nSubtotal\n115200.00\n\nInvoice \n\n0.00\n\nDiscount\n\nTax\n0.00\n\nTotal USD\n115200.00\n\n'

In [42]:
blocks

[{'Item_Number': '',
  'Description': '',
  'Item Details': 'Purchase Order',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': '',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': 'Purchase Order No. 106821',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': '',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': 'Date October 13, 2022',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': 'Page 1',
  'UoM': '',
  'Ordered': '',
  'Unit_Cost': '',
  'Line_Amount': ''},
 {'Item_Number': '',
  'Description': '',
  'Item Details': 'Pro-Tech Sales Company',
  'UoM': '',
  'Ordered': '',
  'Unit_

In [34]:
lines

['Purchase Order',
 '',
 'Purchase Order No. 106821',
 '',
 'Date October 13, 2022',
 'Page 1',
 'Pro-Tech Sales Company ',
 'Inc',
 '2802 Hitchcock ave Downers Grove, IL 60515',
 '',
 '800-766-TECH (8324)   (630) 271-0551    (630) 271-0661 Fax',
 '',
 'Vendor',
 'Ship-to Address',
 '',
 'Lingbo Group Inc.',
 '2802 Hitchcock Ave.',
 '',
 'Carter Ding',
 '',
 '3333 Warrenville Rd. Suite 200',
 'Downers Grove, IL 60515',
 '',
 'Lislie, IL 60532',
 'USA',
 '',
 'Shipment Method',
 'Payment Terms',
 'Buyer',
 'Net 45 days',
 '',
 'Item Number',
 'Description',
 'Req. Date',
 'UofM',
 'Ordered',
 'Unit Cost',
 'Line Amount',
 'B9403-A',
 'Fabricated Saddle (Tenova Core ',
 '',
 'Each',
 '4',
 '28800.0',
 '',
 '115200.00',
 '',
 'ILO1B9344-TS2)',
 '',
 '0',
 '',
 'Material: 310ss',
 '',
 'Subtotal',
 '115200.00',
 '',
 'Invoice ',
 '',
 '0.00',
 '',
 'Discount',
 '',
 'Tax',
 '0.00',
 '',
 'Total USD',
 '115200.00',
 '']

In [ ]:
for line in lines:
    line_strip = line.strip()
    if not inside_table:
        combined = (prev_line + line_strip).replace(" ", "")
        if combined == "Line Amount":
            inside_table = True
            prev_line = ""
            start_marker_found = True
            continue
        #if line_strip == "Tax %":
        #    prev_line = line_strip
        #    continue
        #prev_line = line_strip
        #continue
    if line_strip.startswith(stop_marker):
        break
    current_block.append(line_strip)
    blocks.append(parse_block_protech(current_block))
    current_block = []


In [40]:
def extract_item_blocks_protech(pdf_path: str):
    """Extract item details from PDF table blocks."""
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text_blocks = page.get_text("blocks")
            text_blocks.sort(key=lambda block: (block[1], block[0]))
            for block in text_blocks:
                full_text += block[4] + "\n"
        doc.close()

        # Remove commas in numbers (e.g., 1,000 → 1000)
        full_text = re.sub(r'(?<=\d),(?=\d)', '', full_text)
        lines = full_text.splitlines()
        stop_marker = "Subtotal"

        inside_table = False
        blocks = []
        current_block = []
        #waiting_for_date = False
        prev_line = ""
        start_marker_found = False

        for line in lines:
            line_strip = line.strip()

            if not inside_table:
                combined = (prev_line + line_strip).replace(" ", "")
                if combined == "Line Amount":
                    inside_table = True
                    prev_line = ""
                    start_marker_found = True
                    continue
                #if line_strip == "Tax %":
                #    prev_line = line_strip
                #    continue
                #prev_line = line_strip
                #continue

            if line_strip.startswith(stop_marker):
                break

            current_block.append(line_strip)
            blocks.append(parse_block_protech(current_block))
            current_block = []
                    

        if not start_marker_found:
            return {"error": f"❌ '{os.path.basename(pdf_path)}' cannot convert to CSV, maybe format mismatch."}

        return blocks
    except Exception as e:
        return {"error": str(e)}


def parse_block_protech(block_lines: list) -> dict:
    """Parse individual block of item details."""
    data = {
        "Item_Number": "",
        "Description": "",
        "Item Details": "",
        "UoM": "",
        "Ordered": "",
        "Unit_Cost": "",
        "Line_Amount": ""
    }

    # Delivery Date
    #for i, line in enumerate(block_lines):
    #    if line.strip().startswith("Delivery Date:") and i + 1 < len(block_lines):
    #        data["Delivery Date"] = block_lines[i+1].strip()
    #        break

    # Item Code
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Item Number:") and i + 1 < len(block_lines):
            data["Item_Number"] = block_lines[i+1].strip()
            break

    # Price & Total
    float_lines = [line.strip() for line in block_lines if re.match(r"^\d+\.\d+$", line.strip()) and line.strip() != "0.0000"]
    if float_lines:
        data["Unit_Cost"] = float_lines[0]
        data["Line_Amount"] = max(float_lines, key=lambda x: float(x))

    # Quantity
    try:
        if data["Unit_Cost"] and data["Line_Amount"]:
            data["Ordered"] = str(round(float(data["Line_Amount"]) / float(data["Unit_Cost"]), 4))
    except ZeroDivisionError:
        data["Quantity"] = ""

    # Detect UoM
    for line in block_lines:
        if line.strip() == "Each":
            data["UoM"] = "Each"
            break

    # Description & Item Details
    candidates = []
    ignore_patterns = [r"^\d+\.\d+$", r"^Each$", r"Description", r"Item Number",r"Unit_Cost",r"Line_Amount"]
    for line in block_lines:
        line_strip = line.strip()
        if not any(re.match(p, line_strip) for p in ignore_patterns) and line_strip not in [data["Unit_Cost"], data["Line_Amount"],  data["Item_Number"]]:
            candidates.append(line_strip)

    seen = {}
    duplicates = []
    for line in candidates:
        if line in seen:
            duplicates.append(line)
        else:
            seen[line] = 1

    data["Description"] = " ".join(duplicates).strip()

    for line in block_lines:
        line_strip = line.strip()
        if line_strip in [data["Unit_Cost"], data["Line_Amount"], data["Ordered"],data["UoM"], data["Item_Number"]]:
            continue
        if line_strip.startswith(("Item Code:")):
            continue
        if line_strip in duplicates:
            continue
        try:
            float(line_strip)
            continue
        except ValueError:
            data["Item Details"] = line_strip
            break

    return data



In [31]:
extract_item_blocks_protech(pdf_path)

{'error': "'Delivery Date'"}

In [44]:
pdf_folder = "POs"

# Get the absolute path of the folder
pdf_folder_path = os.path.join("/Users/luzhaolong/Desktop/po_parser", pdf_folder)

# List all PDF files in the folder
pdf_files = glob.glob(os.path.join(pdf_folder_path, "*.pdf"))

# Return the list of PDF file paths
pdf_files  # This will contain all the paths as a list

['/Users/luzhaolong/Desktop/po_parser/POs/PO 100546.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100547.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100350.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100043.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100295.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100327.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100538.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100539.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100298.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100059.pdf',
 '/Users/luzhaolong/Desktop/po_parser/POs/PO 100170.pdf']

In [45]:
pdf_path = pdf_files[0]
doc = fitz.open(pdf_path)
full_text = ""
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    text_blocks = page.get_text("blocks")
    text_blocks.sort(key=lambda block: (block[1], block[0]))
    for block in text_blocks:
        full_text += block[4] + "\n"
doc.close()
# Remove commas in numbers (e.g., 1,000 → 1000)
full_text = re.sub(r'(?<=\d),(?=\d)', '', full_text)

In [46]:
full_text

'▌PURCHASE ORDER\nOriginal\n\nIndustrial Laser Machines, LLC\n\nDocument Number\nDocument Date\nPage\n\n4122 Metric Drive\nSuite 100\nWinter Park\n32792\nFL\n\n1/2\n09/23/2025\nPO100546\n\nCustomer No.\nFederal Tax ID - Business Partner\n\nV0281\n\nYour Reference\n\n250909-01\n\nYour Contact\n\nLINGBO GROUP INC\n\nLloyd\n\n3333\nWARRENVILLE RD\nLISLE\n60532\nIL\nUS\nSTE 200\n\nDelivery Address\n\nIndustrial Laser Machines, LLC\n\n4122 Metric Drive\nSuite 100\nWinter Park\n32792\nFL\nUS\n\nCurrency: $\nDelivery Date:\n12/10/2025\n\nDescription\nQuantity\nUoM\nPrice\nTax %\n Total\nItem Details\nPrice\nTax %Tax %\n\nEach\n25.0000 \n0.0000\n1000.00\nPLATE, BASE, HEAD, UNIVERSAL\n001\n 40\nPLATE, BASE, HEAD, UNIVERSAL\n\nItem Code:\n101581\nItem Code:\nItem Code:\n\nDelivery Date:\n12/10/2025\n\n▌Tax Details\n\nOrder Subtotal:\n$ 1000.00\n\nTax Code\nTax %\nBase Amount\nTax\n\nTotal Before Tax:\n$ 1000.00\n\nEXEMPT\n 0.00\n 1000.00\n 0.0000\n\nTotal Tax Amount:\n$ 0.00\n\n▌Additional Expen

In [16]:

lines = full_text.splitlines()
stop_marker = "▌Tax Details"
inside_table = False
blocks = []
current_block = []
waiting_for_date = False
prev_line = ""
start_marker_found = False


In [7]:
for line in lines: print(line)

▌PURCHASE ORDER
Original

Industrial Laser Machines, LLC

Document Number
Document Date
Page

4122 Metric Drive
Suite 100
Winter Park
32792
FL

1/2
06/19/2025
PO100350

Customer No.
Federal Tax ID - Business Partner

V0281

Your Reference

250611-01

Your Contact

LINGBO GROUP INC

-No Sales Employee-

3333
WARRENVILLE RD
LISLE
60532
IL
US
STE 200

Delivery Address

Industrial Laser Machines, LLC

4122 Metric Drive
Suite 100
Winter Park
32792
FL
US

Currency: $
Delivery Date:
07/17/2025

Description
Quantity
UoM
Price
Tax %
 Total
Item Details
Price
Tax %Tax %

Each
15.5000 
0.0000
155.00
PLATE, SOLENOID & PCB MOUNT, 
SHUTTER
001
 10
Rev A
PLATE, SOLENOID & PCB MOUNT, 
SHUTTER

Item Code:
102219
Item Code:
Item Code:

Delivery Date:
06/19/2025

Each
18.2500 
0.0000
182.50
BRACKET, SENSOR PCB MOUNT, TOP 
ADJUST, SHUTTER
002
 10
Rev A
BRACKET, SENSOR PCB MOUNT, TOP 
ADJUST, SHUTTER

Item Code:
101778-001
Item Code:
Item Code:

Delivery Date:
06/19/2025

▌Tax Details

Order Subtotal:
$ 33

In [17]:
def parse_block(block_lines: list) -> dict:
    """Parse individual block of item details."""
    data = {
        "Item_Code": "",
        "Delivery Date": "",
        "Description": "",
        "Item Details": "",
        "UoM(optional)": "",
        "Quantity": "",
        "Price": "",
        "Total": ""
    }

    # Delivery Date
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Delivery Date:") and i + 1 < len(block_lines):
            data["Delivery Date"] = block_lines[i+1].strip()
            break

    # Item Code
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Item Code:") and i + 1 < len(block_lines):
            data["Item_Code"] = block_lines[i+1].strip()
            break

    # Price & Total
    float_lines = [line.strip() for line in block_lines if re.match(r"^\d+\.\d+$", line.strip()) and line.strip() != "0.0000"]
    if float_lines:
        data["Price"] = float_lines[0]
        data["Total"] = max(float_lines, key=lambda x: float(x))

    # Quantity
    try:
        if data["Price"] and data["Total"]:
            data["Quantity"] = str(round(float(data["Total"]) / float(data["Price"]), 4))
    except ZeroDivisionError:
        data["Quantity"] = ""

    # Detect UoM
    for line in block_lines:
        if line.strip() == "Each":
            data["UoM(optional)"] = "Each"
            break

    # Description & Item Details
    candidates = []
    ignore_patterns = [r"^\d+\.\d+$", r"^Each$", r"Delivery Date:", r"Item Code:"]
    for line in block_lines:
        line_strip = line.strip()
        if not any(re.match(p, line_strip) for p in ignore_patterns) and line_strip not in [data["Price"], data["Total"], data["Delivery Date"], data["Item_Code"]]:
            candidates.append(line_strip)

    seen = {}
    duplicates = []
    for line in candidates:
        if line in seen:
            duplicates.append(line)
        else:
            seen[line] = 1

    data["Description"] = " ".join(duplicates).strip()

    for line in block_lines:
        line_strip = line.strip()
        if line_strip in [data["Price"], data["Total"], data["Quantity"], data["Delivery Date"], "Each", data["Item_Code"]]:
            continue
        if line_strip.startswith(("Delivery Date:", "Item Code:")):
            continue
        if line_strip in duplicates:
            continue
        try:
            float(line_strip)
            continue
        except ValueError:
            data["Item Details"] = line_strip
            break

    return data

In [19]:
for line in lines:
    line_strip = line.strip()
    if not inside_table:
        combined = (prev_line + line_strip).replace(" ", "")
        if combined == "Tax%Tax%":
            inside_table = True
            prev_line = ""
            start_marker_found = True
            continue
        if line_strip == "Tax %":
            prev_line = line_strip
            continue
        prev_line = line_strip
        continue
    if line_strip.startswith(stop_marker):
        break
    current_block.append(line)
    if line_strip.startswith("Delivery Date:"):
        waiting_for_date = True
        continue
    if waiting_for_date:
        if re.match(r"\d{1,2}/\d{1,2}/\d{4}", line_strip):
                    current_block.append(line_strip)
                    blocks.append(parse_block(current_block))
                    current_block = []
                    waiting_for_date = False

    if not start_marker_found:
        print(f"❌ '{os.path.basename(pdf_path)}' cannot convert to CSV, maybe format mismatch.")
blocks

[{'Item_Code': '102219',
  'Delivery Date': '06/19/2025',
  'Description': 'PLATE, SOLENOID & PCB MOUNT, SHUTTER',
  'Item Details': 'Rev A',
  'UoM(optional)': 'Each',
  'Quantity': '10.0',
  'Price': '15.5000',
  'Total': '155.00'},
 {'Item_Code': '101778-001',
  'Delivery Date': '06/19/2025',
  'Description': 'BRACKET, SENSOR PCB MOUNT, TOP ADJUST, SHUTTER',
  'Item Details': 'Rev A',
  'UoM(optional)': 'Each',
  'Quantity': '10.0',
  'Price': '18.2500',
  'Total': '182.50'},
 {'Item_Code': '',
  'Delivery Date': '07/17/2025',
  'Description': 'Industrial Laser Machines, LLC 4122 Metric Drive Suite 100 Winter Park 32792 FL US',
  'Item Details': '▌PURCHASE ORDER',
  'UoM(optional)': '',
  'Quantity': '',
  'Price': '',
  'Total': ''},
 {'Item_Code': '102219',
  'Delivery Date': '06/19/2025',
  'Description': 'Price  PLATE, SOLENOID & PCB MOUNT, SHUTTER',
  'Item Details': 'Description',
  'UoM(optional)': 'Each',
  'Quantity': '10.0',
  'Price': '15.5000',
  'Total': '155.00'},
 {'I

In [ ]:
def parse_block(block_lines: list) -> dict:
    """Parse individual block of item details."""
    data = {
        "Item_Code": "",
        "Delivery Date": "",
        "Description": "",
        "Item Details": "",
        "UoM(optional)": "",
        "Quantity": "",
        "Price": "",
        "Total": ""
    }

    # Delivery Date
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Delivery Date:") and i + 1 < len(block_lines):
            data["Delivery Date"] = block_lines[i+1].strip()
            break

    # Item Code
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Item Code:") and i + 1 < len(block_lines):
            data["Item_Code"] = block_lines[i+1].strip()
            break

    # Price & Total
    float_lines = [line.strip() for line in block_lines if re.match(r"^\d+\.\d+$", line.strip()) and line.strip() != "0.0000"]
    if float_lines:
        data["Price"] = float_lines[0]
        data["Total"] = max(float_lines, key=lambda x: float(x))

    # Quantity
    try:
        if data["Price"] and data["Total"]:
            data["Quantity"] = str(round(float(data["Total"]) / float(data["Price"]), 4))
    except ZeroDivisionError:
        data["Quantity"] = ""

    # Detect UoM
    for line in block_lines:
        if line.strip() == "Each":
            data["UoM(optional)"] = "Each"
            break

    # Description & Item Details
    candidates = []
    ignore_patterns = [r"^\d+\.\d+$", r"^Each$", r"Delivery Date:", r"Item Code:"]
    for line in block_lines:
        line_strip = line.strip()
        if not any(re.match(p, line_strip) for p in ignore_patterns) and line_strip not in [data["Price"], data["Total"], data["Delivery Date"], data["Item_Code"]]:
            candidates.append(line_strip)

    seen = {}
    duplicates = []
    for line in candidates:
        if line in seen:
            duplicates.append(line)
        else:
            seen[line] = 1

    data["Description"] = " ".join(duplicates).strip()

    for line in block_lines:
        line_strip = line.strip()
        if line_strip in [data["Price"], data["Total"], data["Quantity"], data["Delivery Date"], "Each", data["Item_Code"]]:
            continue
        if line_strip.startswith(("Delivery Date:", "Item Code:")):
            continue
        if line_strip in duplicates:
            continue
        try:
            float(line_strip)
            continue
        except ValueError:
            data["Item Details"] = line_strip
            break

    return data

In [9]:
def extract_item_blocks(pdf_path: str):
    """Extract item details from PDF table blocks."""
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text_blocks = page.get_text("blocks")
            text_blocks.sort(key=lambda block: (block[1], block[0]))
            for block in text_blocks:
                full_text += block[4] + "\n"
        doc.close()

        # Remove commas in numbers (e.g., 1,000 → 1000)
        full_text = re.sub(r'(?<=\d),(?=\d)', '', full_text)
        lines = full_text.splitlines()
        stop_marker = "▌Tax Details"

        inside_table = False
        blocks = []
        current_block = []
        waiting_for_date = False
        prev_line = ""
        start_marker_found = False

        for line in lines:
            line_strip = line.strip()

            if not inside_table:
                combined = (prev_line + line_strip).replace(" ", "")
                if combined == "Tax%Tax%":
                    inside_table = True
                    prev_line = ""
                    start_marker_found = True
                    continue
                if line_strip == "Tax %":
                    prev_line = line_strip
                    continue
                prev_line = line_strip
                continue

            if line_strip.startswith(stop_marker):
                break

            current_block.append(line)

            if line_strip.startswith("Delivery Date:"):
                waiting_for_date = True
                continue

            if waiting_for_date:
                if re.match(r"\d{1,2}/\d{1,2}/\d{4}", line_strip):
                    current_block.append(line_strip)
                    blocks.append(parse_block(current_block))
                    current_block = []
                    waiting_for_date = False

        if not start_marker_found:
            return {"error": f"❌ '{os.path.basename(pdf_path)}' cannot convert to CSV, maybe format mismatch."}

        return blocks
    except Exception as e:
        return {"error": str(e)}


def parse_block(block_lines: list) -> dict:
    """Parse individual block of item details."""
    data = {
        "Item_Number": "",
        "Description": "",
        "Item Details": "",
        "UoM": "",
        "Ordered": "",
        "Unit_Cost": "",
        "Line_Amount": ""
    }

    # Delivery Date
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Delivery Date:") and i + 1 < len(block_lines):
            data["Delivery Date"] = block_lines[i+1].strip()
            break

    # Item Code
    for i, line in enumerate(block_lines):
        if line.strip().startswith("Item Number:") and i + 1 < len(block_lines):
            data["Item_Number"] = block_lines[i+1].strip()
            break

    # Price & Total
    float_lines = [line.strip() for line in block_lines if re.match(r"^\d+\.\d+$", line.strip()) and line.strip() != "0.0000"]
    if float_lines:
        data["Unit_Cost"] = float_lines[0]
        data["Line_Amount"] = max(float_lines, key=lambda x: float(x))

    # Quantity
    try:
        if data["Unit_Cost"] and data["Line_Amount"]:
            data["Quantity"] = str(round(float(data["Total"]) / float(data["Price"]), 4))
    except ZeroDivisionError:
        data["Quantity"] = ""

    # Detect UoM
    for line in block_lines:
        if line.strip() == "Each":
            data["UoM"] = "Each"
            break

    # Description & Item Details
    candidates = []
    ignore_patterns = [r"^\d+\.\d+$", r"^Each$", r"Description", r"Item Number",r"Unit_Cost",r"Line_Amount"]
    for line in block_lines:
        line_strip = line.strip()
        if not any(re.match(p, line_strip) for p in ignore_patterns) and line_strip not in [data["Unit_Cost"], data["Line_Amount"], data["Delivery Date"], data["Item_Number"]]:
            candidates.append(line_strip)

    seen = {}
    duplicates = []
    for line in candidates:
        if line in seen:
            duplicates.append(line)
        else:
            seen[line] = 1

    data["Description"] = " ".join(duplicates).strip()

    for line in block_lines:
        line_strip = line.strip()
        if line_strip in [data["Price"], data["Total"], data["Quantity"], data["Delivery Date"], "Each", data["Item_Code"]]:
            continue
        if line_strip.startswith(("Delivery Date:", "Item Code:")):
            continue
        if line_strip in duplicates:
            continue
        try:
            float(line_strip)
            continue
        except ValueError:
            data["Item Details"] = line_strip
            break

    return data

